In [15]:
X_test['fAlpha']= X_test['fAlpha']*data['fAlpha'].std() + data['fAlpha'].mean()  # for this it's more convenient to work with non scaled Alpha values
print(X_test['fAlpha']) 

10357    19.964798
18799    27.708202
1895      4.398689
13945    44.885553
8613     33.457953
           ...    
8484      5.076807
13776    49.203567
3168      9.548524
17152    62.034804
13359    32.729034
Name: fAlpha, Length: 3803, dtype: float64


In [16]:
test_alpha_cut=[]
test_predictions=[]

best_forest = RandomForestClassifier(n_estimators=290, max_depth= 28, max_features=X_train.shape[1],bootstrap=True,max_samples=1.0,random_state=10,n_jobs=-1)
best_forest.fit(X_train,Y_train)

RandomForestClassifier(max_depth=28, max_features=10, max_samples=1.0,
                       n_estimators=290, n_jobs=-1, random_state=10)

In [37]:
gammaness= np.arange(0.8,0.96,0.01)
alpha_cut= np.arange(1,15,1)
     
for alpha in alpha_cut:
    print("\n")
    for g in gammaness:

        test_data = [X_test[ X_test['fAlpha']<alpha], Y_test[X_test['fAlpha']<alpha] ] # only from these samples we can select gamma events, the rest is directly classified as background 
        y_pred_prob = best_forest.predict_proba(test_data[0])[:, 1]
        y_pred_label = np.where(y_pred_prob > g, 1, -1)
        
        selected_gammas= np.sum(y_pred_label==1)
        selected_hadrons= np.sum(y_pred_label==-1) + ( X_test.shape[0]-len(test_data[0])) # samples in alpha cut classified as hadron by the model + all samples previously discarded
    
        total_gammas = np.sum(Y_test==1) # over all the test set (before alpha cut)
        total_hadrons = np.sum(Y_test==-1)
        true_positives = np.sum( (y_pred_label==1) & (test_data[1]==1) )
        false_positives = np.sum( (y_pred_label==1) & (test_data[1]==-1) )
        
        if false_positives != 0:
            epsilon_gamma = true_positives / total_gammas # TPR
            epsilon_hadron = false_positives / total_hadrons # FPR
            Q = epsilon_gamma/np.sqrt(epsilon_hadron)
        else:
            epsilon_hadron= 0
            epsilon_gamma = true_positives / total_gammas
            Q = 'inf'
        
        if Q != 'inf':
            print(f"With alpha cut {alpha} and gammaness {g:.2f}, Q = {Q:.4f}")
        else:
            print(f"With alpha cut {alpha} and gammanesss {g:.2f}, Q = {Q}")
        print(f"True positives {true_positives}; False positives {false_positives}")



With alpha cut 1 and gammanesss 0.80, Q = inf
True positives 11; False positives 0
With alpha cut 1 and gammanesss 0.81, Q = inf
True positives 11; False positives 0
With alpha cut 1 and gammanesss 0.82, Q = inf
True positives 11; False positives 0
With alpha cut 1 and gammanesss 0.83, Q = inf
True positives 11; False positives 0
With alpha cut 1 and gammanesss 0.84, Q = inf
True positives 11; False positives 0
With alpha cut 1 and gammanesss 0.85, Q = inf
True positives 10; False positives 0
With alpha cut 1 and gammanesss 0.86, Q = inf
True positives 9; False positives 0
With alpha cut 1 and gammanesss 0.87, Q = inf
True positives 8; False positives 0
With alpha cut 1 and gammanesss 0.88, Q = inf
True positives 8; False positives 0
With alpha cut 1 and gammanesss 0.89, Q = inf
True positives 8; False positives 0
With alpha cut 1 and gammanesss 0.90, Q = inf
True positives 7; False positives 0
With alpha cut 1 and gammanesss 0.91, Q = inf
True positives 6; False positives 0
With alp

Many of the cases considered return 0 false positives, and thus an "infintite" Q value, indicating that we are succeding in our objective. Between these, of course we prefer the case with the highest number of true positives (to have more gamma events selected to work with). However, there are also cases with a single false positive sample and a higher number of true positives. To avoid choosing the best combination of *alpha cut* and *gammaness* solely on the performance on this specific test set, we run the same algorithm on different test sets and choose as best case the one that returns an "infinite" Q value more times. In case of ties, we choose the *alpha cut* and *gammaness* that allow to find the higher average number of true positives.   

In [59]:
data_rescaled_and_indexed['fAlpha']= data_rescaled_and_indexed['fAlpha']*data['fAlpha'].std() + data['fAlpha'].mean()

In [66]:
N=5
test_predictions=[]
gammaness= np.arange(0.8,0.96,0.01)
alpha_cut= np.arange(2,15,1)

for i in range(1,N+1):
    print('Iteration:',i,'\n')
    
    train_set, test_set = split_train_test_set( data_rescaled_and_indexed, 5)
    X_train, Y_train = split_X_Y(train_set)
    X_test, Y_test = split_X_Y(test_set)
    
    best_forest = RandomForestClassifier(n_estimators=290, max_depth= 28, max_features=X_train.shape[1],bootstrap=True,max_samples=1.0,random_state=10,n_jobs=-1)
    best_forest.fit(X_train,Y_train) 

    for alpha in alpha_cut:
        print('\n')
        for g in gammaness:

            test_data = [ X_test[ X_test['fAlpha']<alpha], Y_test[X_test['fAlpha']<alpha] ] 
            y_pred_prob = best_forest.predict_proba(test_data[0])[:, 1]
            y_pred_label = np.where(y_pred_prob > g, 1, -1)
        
            selected_gammas= np.sum(y_pred_label==1)
            selected_hadrons= np.sum(y_pred_label==-1) + ( X_test.shape[0]-len(test_data[0])) # samples in alpha cut classified as hadron by the model + all samples previously discarded
    
            total_gammas = np.sum(Y_test==1) # over all the test set (before alpha cut)
            total_hadrons = np.sum(Y_test==-1)
            true_positives = np.sum( (y_pred_label==1) & (test_data[1]==1) )
            false_positives = np.sum( (y_pred_label==1) & (test_data[1]==-1) )
        
            if false_positives != 0:
                epsilon_gamma = true_positives / total_gammas # TPR
                epsilon_hadron = false_positives / total_hadrons # FPR
                Q = epsilon_gamma/np.sqrt(epsilon_hadron)
            else:
                epsilon_hadron= 0
                epsilon_gamma = true_positives / total_gammas
                Q = 'inf'
                
            test_predictions.append({
            'iteration': i,   
            'alpha_cut': alpha,
            'gamma': g,
            'test_data': test_data, 
            'pred_probs': y_pred_prob,
            'pred label': y_pred_label,
            'selected gammas': selected_gammas,
            'selected hadrons': selected_hadrons,
            'total gammas': total_gammas,
            'total hadrons': total_hadrons,
            'true positives': true_positives,
            'false positives': false_positives,
            'Q': Q })

            if Q != 'inf':
                print(f"With alpha cut {alpha} and gammaness {g:.2f}, Q = {Q:.4f}")
            else:
                print(f"With alpha cut {alpha} and gammanesss {g:.2f}, Q = {Q}")
            print(f"True positives {true_positives}; False positives {false_positives}")
    

Iteration: 1 



With alpha cut 2 and gammaness 0.80, Q = 1.8404
True positives 339; False positives 8
With alpha cut 2 and gammaness 0.81, Q = 1.8295
True positives 337; False positives 8
With alpha cut 2 and gammaness 0.82, Q = 1.8078
True positives 333; False positives 8
With alpha cut 2 and gammaness 0.83, Q = 1.7861
True positives 329; False positives 8
With alpha cut 2 and gammaness 0.84, Q = 1.7861
True positives 329; False positives 8
With alpha cut 2 and gammaness 0.85, Q = 1.7590
True positives 324; False positives 8
With alpha cut 2 and gammaness 0.86, Q = 1.7535
True positives 323; False positives 8
With alpha cut 2 and gammaness 0.87, Q = 1.8340
True positives 316; False positives 7
With alpha cut 2 and gammaness 0.88, Q = 1.9621
True positives 313; False positives 6
With alpha cut 2 and gammaness 0.89, Q = 2.1151
True positives 308; False positives 5
With alpha cut 2 and gammaness 0.90, Q = 2.3263
True positives 303; False positives 4
With alpha cut 2 and gammaness 0.91, 

In [74]:
test_predictions_df = pd.DataFrame(test_predictions)
test_predictions_df.groupby(['alpha_cut', 'gamma'])['Q'].agg(['mean','std']).reset_index()

,alpha_cut,gamma,mean,std
0,2,0.80,1.813568,0.211618
1,2,0.81,1.858471,0.265411
2,2,0.82,1.867005,0.248918
3,2,0.83,1.855343,0.244921
4,2,0.84,1.932072,0.318225
...,...,...,...,...
203,14,0.91,2.809977,0.311884
204,14,0.92,2.872406,0.227419
205,14,0.93,2.941944,0.302803
206,14,0.94,2.898971,0.256705


In [83]:
grouped_pred = test_predictions_df.groupby(['alpha_cut', 'gamma'])['Q'].agg(['mean','std']).reset_index()
max_mean_row = grouped_pred.loc[grouped_pred['mean'].idxmax()]

max_mean = max_mean_row['mean']
std_dev = max_mean_row['std']

print(f"The best average Q value found is {max_mean:.2f} ± {std_dev:.2f}")
print(f"The best alpha cut found is {max_mean_row['alpha_cut']:.0f}")
print(f"The best gammaness found is {max_mean_row['gamma']:.2f}")


The best average Q value found is 3.09 ± 0.48
The best alpha cut found is 11
The best gammaness found is 0.95
